# Data cleaning
#### Setting the enviorment

In [3]:
!pwd
import numpy as np
import pandas as pd
cd_data = 'data/'
path = cd_data+'On_Time_On_Time_Performance_2017_1.csv'
df = pd.read_csv(path, low_memory=False)

/home/kalen/Projects/CapstoneTwo


# Choosing columns.
While keeping the problem statement in mind, I have selected the columns that will be used to find causalities in flight delays. I hypthesize, that tht most flight delays are due to a chain reaction from one another and orginating from a root delay. 

I will be leaving out coulumns that can be calculated from other columns to save on space and readability.

##### Problem Statment 
*What factors cause airline flight delays in commercial operations and can those factors be used to predict flight delays within 24 hours with an accuracy of at least 90% enabling air traffic to compensate and recover from said delays improving passenger's experience?*



# Summary

1. **DepTime:** Actual Departure Time (local time: hhmm)

2. **TaxiOut:** Taxi Out Time, in Minutes

3. **TaxiIn:** Taxi In Time, in Minutes

4. **ArrTime:** Actual Arrival Time (local time: hhmm)

5. **Cancelled:** Cancelled Flight Indicator (1=Yes)

6. **Diverted:** Diverted Flight Indicator (1=Yes)

7. **AirTime:** Flight Time, in Minutes

8. **Distance:** Distance between airports (miles)

9. **WeatherDelay:** Weather Delay, in Minutes

10. **SecurityDelay:** Security Delay, in Minutes

11. **UniqueCarrier:** Unique Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.

12. ***OriginWac:** Origin Airport, World Area Code

13. **DestWac:** Destination Airport, World Area Code

A list of World Area Codes can be found here: https://en.wikipedia.org/wiki/World_Area_Codes 




In [4]:
#Which values are factors causing a delay?

col = ("""'Year' 'Quarter' 'Month' 'DayofMonth' 'DayOfWeek' 'FlightDate'
 'UniqueCarrier' 'AirlineID' 'Carrier' 'TailNum' 'FlightNum'
 'OriginAirportID' 'OriginAirportSeqID' 'OriginCityMarketID' 'Origin'
 'OriginCityName' 'OriginState' 'OriginStateFips' 'OriginStateName'
 'OriginWac' 'DestAirportID' 'DestAirportSeqID' 'DestCityMarketID' 'Dest'
 'DestCityName' 'DestState' 'DestStateFips' 'DestStateName' 'DestWac'
 'CRSDepTime' 'DepTime' 'DepDelay' 'DepDelayMinutes' 'DepDel15'
 'DepartureDelayGroups' 'DepTimeBlk' 'TaxiOut' 'WheelsOff' 'WheelsOn'
 'TaxiIn' 'CRSArrTime' 'ArrTime' 'ArrDelay' 'ArrDelayMinutes' 'ArrDel15'
 'ArrivalDelayGroups' 'ArrTimeBlk' 'Cancelled' 'CancellationCode'
 'Diverted' 'CRSElapsedTime' 'ActualElapsedTime' 'AirTime' 'Flights'
 'Distance' 'DistanceGroup' 'CarrierDelay' 'WeatherDelay' 'NASDelay'
 'SecurityDelay' 'LateAircraftDelay' 'FirstDepTime' 'TotalAddGTime'
 'LongestAddGTime' 'DivAirportLandings' 'DivReachedDest'
 'DivActualElapsedTime' 'DivArrDelay' 'DivDistance' 'Div1Airport'
 'Div1AirportID' 'Div1AirportSeqID' 'Div1WheelsOn' 'Div1TotalGTime'
 'Div1LongestGTime' 'Div1WheelsOff' 'Div1TailNum' 'Div2Airport'
 'Div2AirportID' 'Div2AirportSeqID' 'Div2WheelsOn' 'Div2TotalGTime'
 'Div2LongestGTime' 'Div2WheelsOff' 'Div2TailNum' 'Div3Airport'
 'Div3AirportID' 'Div3AirportSeqID' 'Div3WheelsOn' 'Div3TotalGTime'
 'Div3LongestGTime' 'Div3WheelsOff' 'Div3TailNum' 'Div4Airport'
 'Div4AirportID' 'Div4AirportSeqID' 'Div4WheelsOn' 'Div4TotalGTime'
 'Div4LongestGTime' 'Div4WheelsOff' 'Div4TailNum' 'Div5Airport'
 'Div5AirportID' 'Div5AirportSeqID' 'Div5WheelsOn' 'Div5TotalGTime'
 'Div5LongestGTime' 'Div5WheelsOff' 'Div5TailNum' 'Unnamed: 109'""")

# cleanining my string of columns. 
col = col.replace("\'", '')
col = col.replace('\n', '')
col = col.split(' ')

elements_to_remove = ['Unnamed:', '109']

for element in elements_to_remove:
    col.remove(element)

col.append('Unnamed: 109') # adding this column back in because it had a space and was split. 



# Removing Columns

In [5]:
# These are the columns chosen for analysis.
keep_col = [
    'UniqueCarrier',
    'ArrTime',
    'DepTime',
    'Distance',
    'WeatherDelay',
    'Diverted',
    'TaxiIn',
    'TaxiOut',
    'Cancellation',
    'AirTime',
    'SecurityDelay', 
    'DestWac',
    'OriginWac']

col_names = set(col).intersection(set(keep_col))
col_names

remove_col = col

df = df[col_names]


# Replacing NaN values
Based on the type of data, I have determined how to handle missing values below. 

In [6]:
#WeatherDelay is a big offender in missing data. Mathmatically I need these missing values to be 0.
df['WeatherDelay'] = df['WeatherDelay'].fillna(0)

# I can assume non-recorded security delays are 0.
df['SecurityDelay'] = df['SecurityDelay'].fillna(0)

# Replacing DepTime and ArrTime Nulls with mean values.

DepTime_mean = df['DepTime'].mean()
df['DepTime'] = df['DepTime'].fillna(DepTime_mean)

ArrTime_mean = df['ArrTime'].mean()
df['ArrTime'] = df['ArrTime'].fillna(ArrTime_mean)

# We can now do the same thing for Taxi Time and AirTime.

TaxiIn_mean = df['TaxiIn'].mean()
df['TaxiIn'] = df['TaxiIn'].fillna(TaxiIn_mean)

TaxiOut_mean = df['TaxiOut'].mean()
df['TaxiOut'] = df['TaxiOut'].fillna(TaxiOut_mean)

TaxiIn_mean = df['AirTime'].mean()
df['AirTime'] = df['AirTime'].fillna(TaxiIn_mean)

In [7]:
len(keep_col) #checking length so I know I did not miss any columns.

13

# Exploring the Data
Using Pandas statistics to get general information about the clean data.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450017 entries, 0 to 450016
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   DestWac        450017 non-null  int64  
 1   UniqueCarrier  450017 non-null  object 
 2   TaxiIn         450017 non-null  float64
 3   OriginWac      450017 non-null  int64  
 4   SecurityDelay  450017 non-null  float64
 5   ArrTime        450017 non-null  float64
 6   WeatherDelay   450017 non-null  float64
 7   Distance       450017 non-null  float64
 8   Diverted       450017 non-null  float64
 9   DepTime        450017 non-null  float64
 10  TaxiOut        450017 non-null  float64
 11  AirTime        450017 non-null  float64
dtypes: float64(9), int64(2), object(1)
memory usage: 41.2+ MB


In [9]:
df.isnull().sum() #Checking for NaNs.

DestWac          0
UniqueCarrier    0
TaxiIn           0
OriginWac        0
SecurityDelay    0
ArrTime          0
WeatherDelay     0
Distance         0
Diverted         0
DepTime          0
TaxiOut          0
AirTime          0
dtype: int64

In [10]:
df.head()

,DestWac,UniqueCarrier,TaxiIn,OriginWac,SecurityDelay,ArrTime,WeatherDelay,Distance,Diverted,DepTime,TaxiOut,AirTime
0,81,AA,5.0,36,0.0,1842.0,0.0,1773.0,0.0,1616.0,17.0,244.0
1,81,AA,6.0,36,0.0,1821.0,0.0,1773.0,0.0,1614.0,13.0,228.0
2,81,AA,2.0,36,0.0,1826.0,0.0,1773.0,0.0,1611.0,17.0,236.0
3,81,AA,3.0,36,0.0,1929.0,0.0,1773.0,0.0,1656.0,18.0,252.0
4,81,AA,4.0,36,0.0,1858.0,0.0,1773.0,0.0,1632.0,17.0,245.0


In [11]:
df.describe()

,DestWac,TaxiIn,OriginWac,SecurityDelay,ArrTime,WeatherDelay,Distance,Diverted,DepTime,TaxiOut,AirTime
count,450017.000000,450017.000000,450017.000000,450017.000000,450017.000000,450017.000000,450017.000000,450017.000000,450017.000000,450017.000000,450017.000000
mean,55.815351,7.694634,55.814031,0.017313,1487.332958,0.731381,852.125140,0.003302,1333.947392,17.066131,118.427863
std,27.088053,6.426908,27.091711,1.349893,518.397960,13.031866,617.376659,0.057369,490.237719,9.850732,73.665253
min,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,31.000000,0.000000,1.000000,1.000000,7.000000
25%,34.000000,5.000000,34.000000,0.000000,1113.000000,0.000000,391.000000,0.000000,927.000000,11.000000,63.000000
50%,52.000000,6.000000,52.000000,0.000000,1511.000000,0.000000,687.000000,0.000000,1333.947392,15.000000,102.000000
75%,82.000000,9.000000,82.000000,0.000000,1916.000000,0.000000,1096.000000,0.000000,1734.000000,20.000000,148.000000
max,93.000000,413.000000,93.000000,653.000000,2400.000000,1934.000000,4983.000000,1.000000,2400.000000,176.000000,704.000000


In [12]:
df.corr()

,DestWac,TaxiIn,OriginWac,SecurityDelay,ArrTime,WeatherDelay,Distance,Diverted,DepTime,TaxiOut,AirTime
DestWac,1.000000,0.024216,0.453457,0.000349,0.018917,0.003819,0.001942,1.142862e-02,0.032443,-0.001789,6.173559e-02
TaxiIn,0.024216,1.000000,-0.011613,-0.000444,-0.006344,0.006760,0.098212,1.399568e-02,-0.035682,0.029235,1.067723e-01
OriginWac,0.453457,-0.011613,1.000000,-0.000716,0.021270,-0.003155,0.002189,-9.701746e-03,-0.018189,-0.029668,-6.566115e-02
SecurityDelay,0.000349,-0.000444,-0.000716,1.000000,-0.001520,-0.000606,0.005597,-7.382078e-04,0.008471,0.004477,6.014892e-03
ArrTime,0.018917,-0.006344,0.021270,-0.001520,1.000000,-0.017883,0.021497,-1.019539e-02,0.644823,-0.022024,2.406819e-02
WeatherDelay,0.003819,0.006760,-0.003155,-0.000606,-0.017883,1.000000,-0.000033,-3.230360e-03,-0.004017,0.078007,3.560398e-03
Distance,0.001942,0.098212,0.002189,0.005597,0.021497,-0.000033,1.000000,2.560659e-02,-0.023991,0.040537,9.647253e-01
Diverted,0.011429,0.013996,-0.009702,-0.000738,-0.010195,-0.003230,0.025607,1.000000e+00,0.001799,0.016950,4.782207e-17
DepTime,0.032443,-0.035682,-0.018189,0.008471,0.644823,-0.004017,-0.023991,1.799018e-03,1.000000,-0.048391,-2.909207e-02
TaxiOut,-0.001789,0.029235,-0.029668,0.004477,-0.022024,0.078007,0.040537,1.695029e-02,-0.048391,1.000000,5.434242e-02


# Writing the Cleaned Data

In [14]:
df.to_csv(cd_data+'flight_delays.csv', index=False) # Saving clean data.